In [ ]:
from __future__ import print_function

import glob
from itertools import chain

import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

import random

In [ ]:
print(f"Torch: {torch.__version__}")

In [ ]:
# Training settings
batch_size = 64
epochs = 20
lr = 5e-4
gamma = 0.8
seed = 42
num_classes = 1
device = 'cuda'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

The same preprocesssing that I used to create the dataset. This will be used on the training data before we submit to the leaderboard

In [ ]:
#The Code from: https://www.kaggle.com/ratthachat/aptos-updated-albumentation-meets-grad-cam
import cv2

def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

Lets start by transforming the test images in the same way as the training images.

In [ ]:
inPath = '../input/aptos2019-blindness-detection/test_images'
  
# path of the folder that will contain the modified image
try:
    os.mkdir("test_images_transformed")
except:
    print("path already exists")

outPath ="test_images_transformed"

for imagePath in tqdm(os.listdir(inPath)):
    # imagePath contains name of the image 
    inputPath = os.path.join(inPath, imagePath)

    image = cv2.imread(inputPath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (224, 224))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur( image , (0,0) , 30) ,-4 ,128)

    fullOutPath = os.path.join(outPath, imagePath)
    cv2.imwrite(fullOutPath, image)

In [ ]:
train_dir = '../input/custom-transform-blindness-2019/train_images_transformed'
test_dir = './test_images_transformed'

In [ ]:
train_list = glob.glob(os.path.join(train_dir,'*.*'))
test_list = glob.glob(os.path.join(test_dir, '*.png'))

In [ ]:
print(f"Train Data: {len(train_list)}")
print(f"Test Data: {len(test_list)}")

loading the labels

In [ ]:
df_train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_train_old = pd.read_csv("../input/resized-2015-2019-blindness-detection-images/labels/trainLabels15.csv")
df_train_old = df_train_old.rename({"image" : "id_code", "level" : "diagnosis"}, axis=1)
df_train = df_train.append(df_train_old).reset_index(drop=True)

labels = df_train['diagnosis'].values
label_lookup = df_train.set_index('id_code')

df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
class_weights = df_train['diagnosis'].value_counts()
dfs = [df_train[df_train['diagnosis'] == i].sample(class_weights[4]) for i in range(5)]
resampled = pd.concat(dfs, axis = 0)

In [ ]:
resampled.diagnosis.value_counts()

In [ ]:
new_train_list = (train_dir + '/' + resampled['id_code'].apply(lambda x: x + ('.jpg' if '_' in x else '.png'))).values
new_train_list

encoding

In [ ]:
y_train = pd.get_dummies(df_train['diagnosis']).values

print(y_train.shape)
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 4] = y_train[:, 4]

for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])

print("Original y_train:", y_train.sum(axis=0))
print("Multilabel version:", y_train_multi.sum(axis=0))

In [ ]:
y_train_multi

In [ ]:
get_index = lambda x : df_train[df_train.id_code == x].index[0]
y_train_multi[get_index('0a4e1a29ffff')]

Train Validaition split. This validation is only used to monitor the model's performance. The true test set for the leaderboard is a secret, and will be run without us having access to it.

In [ ]:
train_list, valid_list = train_test_split(new_train_list, test_size=0.05, random_state=seed)

In [ ]:
print(len(train_list))
print(len(valid_list))

In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
#         transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

In [ ]:
class Blindness2019(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = label_lookup.loc[img_path.split("/")[-1].split(".")[0]][0]
#         label = torch.tensor(label).to(torch.float32)
        image_id = img_path.split("/")[-1].split(".")[0]
#         label = y_train_multi[get_index(image_id)]
#         label = y_train_multi[random.randint(0,3000)]
        return img_transformed, label

class Blindness2019Test(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
            
        return img_transformed

In [ ]:
train_data = Blindness2019(train_list, transform=train_transforms)
valid_data = Blindness2019(valid_list, transform=test_transforms)
test_data = Blindness2019Test(test_list, transform=test_transforms)

Creating data loader with the batch size

In [ ]:
train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=1, shuffle=False)

In [ ]:
print(len(train_data), len(train_loader))
print(len(valid_data), len(valid_loader))

Uncomment one of the below cells to pick the model you want to run
All the models are in the same notebook, but it would take too long to run them all. So instead, pick one, and uncomment it! This will be the model variable that the training loop uses

Efficientnet

In [ ]:
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_name('efficientnet-b0')
model.load_state_dict(torch.load('../input/efficientnet-pytorch/efficientnet-b0-08094119.pth'))
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, 5)
# model.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1,1))
# model._fc = nn.Sequential(
#                 nn.ReLU(),
#                 nn.Linear(in_features=in_features, out_features=128, bias=True),
#                 nn.ReLU(),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             )
model = model.to(device)

In [ ]:
! pip install ../input/vit-pytorch/Pillow-8.4.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
! pip install ../input/vit-pytorch/einops-0.3.2-py3-none-any.whl
! pip install ../input/vit-pytorch/numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
! pip install ../input/vit-pytorch/torch-1.10.0-cp37-cp37m-manylinux1_x86_64.whl
! pip install ../input/vit-pytorch/torchvision-0.11.1-cp37-cp37m-manylinux1_x86_64.whl
! pip install ../input/vit-pytorch/typing_extensions-4.0.1-py3-none-any.whl
! pip install ../input/vit-pytorch/vit_pytorch-0.24.3-py3-none-any.whl
from vit_pytorch.efficient import ViT

! pip install ../input/linformer/linformer-0.2.1-py3-none-any.whl
! pip install ../input/linformer/torch-1.10.0-cp37-cp37m-manylinux1_x86_64 (1).whl
! pip install ../input/linformer/typing_extensions-4.0.1-py3-none-any (1).whl
from linformer import Linformer

Linformer ViT

In [ ]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=32,
    heads=16,
    k=64
)

v = ViT(
    dim=32,
    image_size=224,
    patch_size=16,
    num_classes=5,
    transformer=efficient_transformer,
    channels=3,
)

# model = nn.Sequential(
#                 v,
# #                 nn.Dropout(p=.5),
#                 nn.Linear(in_features=256, out_features=128, bias=True),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             ).to(device)

Deep ViT

In [ ]:
# import torch
# from vit_pytorch.deepvit import DeepViT

# v = DeepViT(
#     image_size = 224,
#     patch_size = 32,
#     num_classes = 256,
#     dim = 1024,
#     depth = 6,
#     heads = 16,
#     mlp_dim = 2048,
#     dropout = 0.1,
#     emb_dropout = 0.1
# ).to(device)


# model = nn.Sequential(
#                 v,
# #                 nn.Dropout(p=.5),
#                 nn.Linear(in_features=256, out_features=128, bias=True),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             ).to(device)